In [1]:
# Import the dependencies

import string
import pandas as pd
from sklearn.utils import shuffle

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from textblob import TextBlob, Word
import s

from multiprocessing import Process, Value, Pool

import re

# Neural Network machine learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Load the data

#### First dataset

In [3]:
# Load the data from the first dataset

ds1_real_news = pd.read_csv("data/1/True.csv")
ds1_fake_news = pd.read_csv("data/1/Fake.csv")

#### Second Dataset

In [4]:
# Load the data from the second dataset

ds2_news = pd.read_csv("data/2/news.csv")

### Analysis and transformations

#### First dataset

In [5]:
# Show the first five rows of the dataset composed of ds1_real_news

ds1_real_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
# Check the dimensions of the ds1_real_news dataframe

ds1_real_news.shape

(21417, 4)

In [7]:
# Check for null values in the ds1_real_news dataframe

ds1_real_news.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [8]:
# Show the first five rows of the dataset composed of ds1_fake_news

ds1_fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [9]:
# Check the dimensions of the ds1_fake_news dataframe

ds1_fake_news.shape

(23481, 4)

In [10]:
# Check for null values in the ds1_fake_news dataframe

ds1_fake_news.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [11]:
# Add a label column to the ds1_real_news dataframe
ds1_real_news["label"] = "REAL"

# Add a label column to the ds1_fake_news dataframe
ds1_fake_news["label"] = "FAKE"

# Merge the two datasets vertically
ds1_news = ds1_real_news.append(ds1_fake_news, ignore_index=True)
ds1_news.shape

(44898, 5)

In [12]:
ds1_news.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",REAL
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",REAL
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",REAL
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",REAL
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",REAL


In [13]:
# Remove the date and subject columns because it does not contain any useful information
ds1_news.drop(["date","subject"],axis=1,inplace=True)
ds1_news.columns

Index(['title', 'text', 'label'], dtype='object')

#### Second Dataset

In [14]:
# Load the data from the second dataset

ds2_news = pd.read_csv("data/2/news.csv")

In [15]:
# Show the first five rows of the second dataset composed of both real and fake news

ds2_news.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [16]:
# Check the dimensions of the all_news_2 dataframe

ds2_news.shape

(6335, 4)

In [17]:
# Check for null values in the all_news_2 dataframe

ds2_news.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [18]:
# Drop the Unnamed: 0 column from the news dataframe

ds2_news.drop(columns=["Unnamed: 0"], inplace=True)
ds2_news.columns

Index(['title', 'text', 'label'], dtype='object')

In [19]:
# Merge the first dataset with the second dataset

all_news = ds1_news.append(ds2_news, ignore_index=True)
all_news.head()

,title,text,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,REAL
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,REAL
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,REAL
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,REAL
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,REAL


In [20]:
# Show the dimensions of the merged dataset
all_news.shape

(51233, 3)

# Combine the title column and text column

all_news["article"] = all_news.apply(lambda x:x["title"] + x["text"], axis=1)
all_news.head()

# Drop the title column and text column

all_news.drop(["title","text"], axis=1, inplace=True)
all_news.head()

In [21]:
# shuffle the rows of the all_news dataframe 10 times

for i in range(10):
    all_news = shuffle(all_news)

In [22]:
all_news.head()

,title,text,label
6735,Many Russian officials have good ties with Til...,MOSCOW (Reuters) - Not only Russian President ...,REAL
22279,Latest House Vote Proves Republicans Are Read...,With the release of the email chain that prove...,FAKE
44855,"BOILER ROOM – EP #45 – Horror Hotel, Trump Gat...",Tune in to the Alternate Current Radio Network...,FAKE
9985,Police arrest 400 at U.S. Capitol in protest o...,WASHINGTON (Reuters) - Police arrested more th...,REAL
30311,‘Family Values’ Republicans Introduce A Bill ...,You have to love the Republican party. They ca...,FAKE


### Machine Learning

First, the two datasets need to be merged. This would be done in two steps:
- Add a **label** column to each dataset. The column will contain FAKE in the fake dataset and REAL in the real dataset.
- Vertically merge the dataframes, adding the true dataset to the end of the fake dataset.

In order to determine which words and sentences to use in the machine learning algorithm, the title and text columns have to be parsed into their component words. The words are then transformed into a simpler form, either by stemming, which involves truncating words (more or less), or lemmatization, which involves mapping each word to its grammatical source, eg/ bigger and biggest would be transformed to big, and see and saw would be transformed to see. The remaining words are then vectorized and then the vectorized dataset split up into a training set, to train a classification machine learning algorithm, and a test set, to test the predictions of the generated model.



### Processing text column

In [23]:
def to_lower_case(df,column):
    
    lowercase = df[column].apply(lambda x: x.lower())
    
    return lowercase

In [24]:
def join_chars(text):
    
    if not isinstance(text,str):
        print("This")
    # print(f"type: {type(text)} \noffending text --> {text}")
    
    joined = "".join([char for char in text if char not in string.punctuation])
    return joined
    # print(f"============================================> \n\n {joined}")                

In [25]:
def remove_punctuation(df,column):
    try:
        df["lc_rp"] = df[column].apply(lambda x: join_chars(x))
    except:
        print(f"Exception: {df}")
        
    return df

In [26]:
def remove_digits(df, column):
    digits = '0123456789'
    
    no_digits = df[column].apply(lambda l: "".join([x for x in l if str(x) not in digits]))
    
    return no_digits

In [27]:
# Insert a space after every period signifying the end of a sentence. Also insert a space between every comma.

def period_space(df, column):
    
    period_space = df[column].apply(lambda x: print(re.subn(r'(\s\w+\.)(\w+\s)', '\\1 \\2', x, flags=re.IGNORECASE)))

    return period_space

In [28]:
def stop_words():
    sw1 = stopwords.words("english")
    sw2 = open("nlp/stop_words_english.txt").read().splitlines()

    return set(sw1 + sw2)

In [59]:
Word("youngest").lemmatize()

'youngest'

In [30]:
def remove_stopwords(df,column):
    text_no_sw = df[column].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words()]))
    
    return text_no_sw

In [31]:
stemmer = SnowballStemmer("english")

def stem(df, column):
    stemmed = df[column].apply(lambda l: " ".join(set([stemmer.stem(x) for x in l.split()])))
    
    return stemmed

In [50]:
stemmer.stem("arabic")

'arab'

In [33]:
# lemmatizer = WordNetLemmatizer()

def lemmatize(df,column):
    lemmatized = df[column].apply(lambda l: " ".join(set([Word(x).lemmatize() for x in l.split()])))
    
    return lemmatized

In [51]:
def clean_articles(args):
    
    df, column = args[0:2]
    df["lowercase"] = df[column].apply(lambda x: x.lower())
    df.drop(columns=[column],inplace=True)
    # df = remove_punctuation(df,"_".join([column,"lowercase"])
    df["add_spaces"] = df["lowercase"].apply(lambda x: re.subn(r'(\s\w+\.)(\w+\s)', '\\1 \\2', x, flags=re.IGNORECASE)[0])
    
    df.drop(columns=["lowercase"],inplace=True)
    df["no_digits"] = remove_digits(df,"add_spaces")

    df.drop(columns=["add_spaces"],inplace=True)
    df["text_no_sw"] = remove_stopwords(df,"no_digits")
    
    df.drop(columns=["no_digits"],inplace=True)
    
#     df["lemmatized"] = lemmatize(df,"text_no_sw")
#     df.drop(columns=["text_no_sw"],inplace=True)

    df["stemmed"] = stem(df,"text_no_sw")
    df.drop(columns=["text_no_sw"],inplace=True)
 
    df["lemmatized"] = lemmatize(df,"stemmed")
    df.drop(columns=["stemmed"],inplace=True)
    
    return df

In [35]:
all_news.reset_index(inplace=True)

In [36]:
all_news.shape[0]

51233

In [37]:
all_news = all_news.iloc[:1000]

all_news = pd.DataFrame({
    "label": ["FAKE",'REAL'],
    "article": ["THEre ArE4 twenTY5 isn't","1074bradley AVE"]
})
clean_articles([all_news,"article"])

In [52]:
# Parallellize processing - Use 5 cores
# if __name__ == "__main__":

batch_size = 10000
rows = all_news.shape[0]
num_proc = rows // batch_size

args_list = []

for n in range(num_proc):
    start,end = n*batch_size, (n+1)*batch_size
    args = ( all_news.iloc[start:end], "text")
    args_list.append(args)

# print(f"rows_left = {rows % batch_size}")

if rows % batch_size > 0:
    start,end = (num_proc) * batch_size, rows
    args = ( all_news.iloc[start:end], "text")
    args_list.append(args)
    num_proc += 1

# print(num_proc)

p = Pool(processes=6)

data = p.map(clean_articles, args_list)
p.close()

cleaned = pd.DataFrame()

for df in data:
    cleaned = cleaned.append(df)


In [53]:
cleaned

,index,title,label,lemmatized
0,6735,Many Russian officials have good ties with Til...,REAL,"american activ moscow tillerson, russian tille..."
1,22279,Latest House Vote Proves Republicans Are Read...,FAKE,"democrat markup mix down, block enforc heel wa..."
2,44855,"BOILER ROOM – EP #45 – Horror Hotel, Trump Gat...",FAKE,"verit of. stewart serial media-maniacs, intere..."
3,9985,Police arrest 400 at U.S. Capitol in protest o...,REAL,"capitol, spring, democrat next.” activist prot..."
4,30311,‘Family Values’ Republicans Introduce A Bill ...,FAKE,church gift. specul of. kerfuffl wikipedia und...
...,...,...,...,...
995,12113,Libyan coast guard rescues more than 250 migra...,REAL,smuggl coastal accord illeg captain (on migran...
996,29686,Hillary Gives Rubio A New Name After He Says ...,FAKE,"democrat rais saturday choic capture. know, cr..."
997,30064,WATCH: This Hilarious Video Is The Only Way P...,FAKE,"rest establish democrat poor me, ha illeg yout..."
998,4771,Factbox: Political risk outlook calendar for T...,REAL,calendar requir budget dec. influenc june rece...


In [57]:
lemmatizer.lemmatize("younger")

NameError: name 'lemmatizer' is not defined

### Machine Learning

Since this dataset contains FAKE and REAL news articles, no merge step is required.

The remainging steps, including parsing, stemming or lemmatization, vectorization, and then classification machine learning, that were mentioned above, are all steps that would be applied to process this dataset. 

In [55]:
# Feature Matrix creation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()

X = tfidf_vect.fit_transform(cleaned.lemmatized)
# X = tfidf_vect.fit_transform(cleaned.stemmed)

In [56]:
vocab = list(tfidf_vect.vocabulary_)
vocab_set = set(vocab)
file = open("vocab_set.txt","w")
for word in vocab_set:
    file.writelines("".join([word,"\n"]))
file.close()

In [41]:
vocab_set

{'threatsthe',
 'httpstcopxwjefjmp',
 'diseasemodifying',
 'subjectone',
 'wannabefascistinchief',
 'anonymizing',
 'wrotehasen',
 'vulnerablewatchvia',
 'ratehike',
 'division',
 'beecher',
 'downislamabad',
 'pictwittercomntqtovipsl',
 'canineaccording',
 'homeseveral',
 'foodstsnn',
 'pulloutmoscow',
 'ficer',
 'speechdont',
 'accomplishmentseven',
 'avoidability',
 'farren',
 'houseaboard',
 'meetingmuch',
 'backroads',
 'reusch',
 'hyperbriefed',
 'questionday',
 'hickory',
 'retro',
 'zirconium',
 'claimsome',
 'transportationthe',
 'communityon',
 'abroadoslowashington',
 'cannier',
 'instagaffe',
 'symbology',
 'httpstcoooxtmxra',
 'sabato',
 'opinionspictwittercomtxjaquxfg',
 'movadef',
 'dogging',
 'lostit',
 'deena',
 'gobblin',
 'taodehaas',
 'sleet',
 'pictwittercomalwkhvyu',
 'repercussionsimage',
 'trumbullmothers',
 'abcdisney',
 'tonton',
 'undertone',
 'domic',
 'pollmonitor',
 'espncolin',
 'unclearhowever',
 'taxcollection',
 'pseudonymous',
 'formulation',
 'consen

In [1]:
cleaned.loc[cleaned["lemmatized"].str.contains("ralat")]["lemmatized"]

NameError: name 'cleaned' is not defined

In [40]:
cleaned["len"] = cleaned["lemmatized"].apply(lambda x: len(x))
max(cleaned["len"])

24239

In [37]:
cleaned.loc[cleaned["len"]==24239]["lemmatized"]

9370    edmund blip devalued privacy ugly spew santiag...
Name: lemmatized, dtype: object

In [44]:
# Target Series creation

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(cleaned.label).reshape(-1,1)

In [45]:
y

array([[0],
       [0],
       [1],
       ...,
       [0],
       [1],
       [0]])

In [48]:
# Define Sequential model with 3 layers
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(600000, input_shape=(274505,), activation="relu"))
model.add(tf.keras.layers.Dense(6000), activation="relu")
model.add(tf.keras.layers.Dense(1), activation="sigmoid")

model.compile()




ResourceExhaustedError: OOM when allocating tensor with shape[274505,600000] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:RandomUniform]

## APIs

The following three APIs will be used to stream news articles:

* Mediastack API (https://api.mediastack.com)
* Newsapi API (https://newsapi.org)
* NY Times API (https://api.nytimes.com)

For each of the APIs, there is a link (URL) which  is used to retrieve articles. To insert the articles into an SQL database, the response, which comprises the retrieved articles, has to be split up into individual articles which, using prepared statements, are inserted into the database. The process is automated by creating a continuously-running Python app to periodically (hourly/daily/weekly) retrieve apps from the news sites and populate the database. 

In [36]:
cleaned.iloc[30000:40000]

,index,label,article,lowercase,lc_rp,text_no_sw
30000,31529,FAKE,TRUMP’S DOJ MAKES Announcement on Anti-Gun Oba...,trump’s doj makes announcement on anti-gun oba...,trump’s doj makes announcement on antigun obam...,trump’s doj makes announcement antigun obamaer...
30001,28802,FAKE,WATCH: President Obama Just Landed In Cuba An...,watch: president obama just landed in cuba an...,watch president obama just landed in cuba and...,watch president obama landed cuba gop going cr...
30002,36825,FAKE,(VIDEO) CHICAGO THUG PRESIDENT GOES AFTER TRUM...,(video) chicago thug president goes after trum...,video chicago thug president goes after trump ...,video chicago thug president goes trump “outra...
30003,982,REAL,First charges filed in Russia probe led by U.S...,first charges filed in russia probe led by u.s...,first charges filed in russia probe led by us ...,first charges filed russia probe led us specia...
30004,29797,FAKE,Michele Bachmann Confuses Direct-To-DVD Movie...,michele bachmann confuses direct-to-dvd movie...,michele bachmann confuses directtodvd movie p...,michele bachmann confuses directtodvd movie pl...
...,...,...,...,...,...,...
39995,16633,REAL,Saudi foreign minister says backs Trump's stan...,saudi foreign minister says backs trump's stan...,saudi foreign minister says backs trumps stanc...,saudi foreign minister says backs trumps stanc...
39996,26890,FAKE,WATCH: Fox News Host Vies For Dumbest On Netw...,watch: fox news host vies for dumbest on netw...,watch fox news host vies for dumbest on netwo...,watch fox news host vies dumbest network talki...
39997,11563,REAL,"After Reuters arrests, some Myanmar reporters ...","after reuters arrests, some myanmar reporters ...",after reuters arrests some myanmar reporters f...,reuters arrests myanmar reporters fear couldve...
39998,36299,FAKE,What Gives? Saudi King Spends A Fortune In 201...,what gives? saudi king spends a fortune in 201...,what gives saudi king spends a fortune in 2014...,gives saudi king spends fortune 2014 gifts oba...
